In [2]:
import numpy as np
import sleap_io as sio
import re
import pandas as pd

from brainglobe_registration.elastix.register import (
    run_registration
)
from brainglobe_registration.utils.utils import open_parameter_file
from skimage.color import rgb2gray
from pathlib import Path


In [15]:
video_path = '../21Jan_007.mp4'
video = sio.load_video(video_path)

video_subsample = video[0:500]
video_subsample.shape

(500, 1080, 1920, 3)

In [74]:
file_path = (
    Path.home() / "NIU_dev" / "bg-elastix"
    / "brainglobe_registration"
    / "parameters"
    / "brainglobe_registration"
    / "affine.txt"
)

parameter_lists = [("affine", open_parameter_file(file_path))]
parameter_lists[0][1]["FixedInternalImagePixelType"] = ["float"]
parameter_lists[0][1]["MovingInternalImagePixelType"] = ["float"]
parameter_lists[0][1]["MaximumNumberOfIterations"] = ["300"]
parameter_lists[0][1]["Transform"] = ["EulerTransform"]

In [80]:
from tqdm.notebook import tqdm

output_file = "out_euler_10.csv"

with open(output_file, "w") as f:
    f.write("theta,tx,ty\n")

for i in tqdm(range(1, 10)):
    fixed_gray = rgb2gray(video_subsample[i-1])
    moving_gray = rgb2gray(video_subsample[i])
    parameters = run_registration(
        fixed_gray,
        moving_gray,
        parameter_lists,
        filter_images=False
    )

    # Regular expression to find the TransformParameters line
    pattern = r'\(TransformParameters ([\d\.\-e ]+)\)'
    input_string = str(parameters)
    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    if match:
        # Extract the numbers and convert them to floats
        transform_parameters = list(map(float, match.group(1).split()))
        with open(output_file, "a") as f:
            f.write(",".join(map(str, transform_parameters)))
            f.write("\n")
    else:
        print("TransformParameters not found")

  0%|          | 0/9 [00:00<?, ?it/s]

In [51]:
str(parameters)

'ParameterObject (0x600004d5caa0)\n  RTTI typeinfo:   elastix::ParameterObject\n  Reference Count: 1\n  Modified Time: 19673950\n  Debug: Off\n  Object Name: \n  Observers: \n    none\nParameterMap 0: \n  (CenterOfRotationPoint 959.5 539.5)\n  (CompressResultImage "false")\n  (DefaultPixelValue 0)\n  (Direction 1 0 0 1)\n  (FinalBSplineInterpolationOrder 3)\n  (FixedImageDimension 2)\n  (FixedInternalImagePixelType "float")\n  (HowToCombineTransforms "Compose")\n  (Index 0 0)\n  (InitialTransformParameterFileName "NoInitialTransform")\n  (MovingImageDimension 2)\n  (MovingInternalImagePixelType "float")\n  (NumberOfParameters 6)\n  (Origin 0 0)\n  (ResampleInterpolator "FinalBSplineInterpolator")\n  (Resampler "DefaultResampler")\n  (ResultImageFormat "tiff")\n  (ResultImagePixelType "double")\n  (Size 1920 1080)\n  (Spacing 1 1)\n  (Transform "AffineTransform")\n  (TransformParameters 1.00025 1.48686e-06 9.15803e-05 1.00018 1.22378 0.384018)\n  (UseDirectionCosines "true")\n'

In [48]:
str(parameters)

'ParameterObject (0x600004d5c640)\n  RTTI typeinfo:   elastix::ParameterObject\n  Reference Count: 1\n  Modified Time: 19419312\n  Debug: Off\n  Object Name: \n  Observers: \n    none\nParameterMap 0: \n  (CenterOfRotationPoint 959.5 539.5)\n  (CompressResultImage "false")\n  (DefaultPixelValue 0)\n  (Direction 1 0 0 1)\n  (FinalBSplineInterpolationOrder 3)\n  (FixedImageDimension 2)\n  (FixedInternalImagePixelType "float")\n  (HowToCombineTransforms "Compose")\n  (Index 0 0)\n  (InitialTransformParameterFileName "NoInitialTransform")\n  (MovingImageDimension 2)\n  (MovingInternalImagePixelType "float")\n  (NumberOfParameters 3)\n  (Origin 0 0)\n  (ResampleInterpolator "FinalBSplineInterpolator")\n  (Resampler "DefaultResampler")\n  (ResultImageFormat "tiff")\n  (ResultImagePixelType "double")\n  (Size 1920 1080)\n  (Spacing 1 1)\n  (Transform "EulerTransform")\n  (TransformParameters 8.22838e-05 1.26321 0.410233)\n  (UseDirectionCosines "true")\n'

In [73]:
cos_value = 1.00025

np.cos(8.22838e-05)

np.float64(0.9999999966146881)

In [81]:

df = pd.read_csv(output_file)
## Add the first frame row
df = pd.concat([pd.DataFrame({
    'theta': [0],
    'tx': [0],
    'ty': [0]
}), df], ignore_index=True)
df.head()

,theta,tx,ty
0,0.000000,0.00000,0.000000
1,0.000042,1.07159,0.449169
2,0.000014,1.20212,0.420697
3,0.000096,1.08352,0.401244
4,0.000076,1.13084,0.443089


In [82]:
df['tx_sum'] = df['tx'].cumsum()
df['ty_sum'] = df['ty'].cumsum()

In [83]:
df['tx_sum_int'] = df['tx_sum'].round(0).astype(int)
df['ty_sum_int'] = df['ty_sum'].round(0).astype(int)

x_min = df['tx_sum_int'].min()
x_max = df['tx_sum_int'].max()
y_min = df['ty_sum_int'].min()
y_max = df['ty_sum_int'].max()
print(x_min, x_max, y_min, y_max)

0 11 0 4


In [84]:
height, width = video_subsample.shape[1:3]

# total_height = y_max - y_min + height
# total_width = x_max - x_min + width
total_height = 2500
total_width = 3000

print(total_height, total_width)

2500 3000


In [85]:
import numpy as np
from skimage.transform import rotate

fused_image = np.zeros((total_height, total_width, 3), dtype=np.uint8)

for i in range(10 - 1, 0, -1):
    # Rotation around the center of the image
    adjusted_image = rotate(video_subsample[i], df['theta'][i], resize=True)
    x = df['tx_sum_int'][i] - x_min
    y = df['ty_sum_int'][i] - y_min
    fused_image[y:y+height, x:x+width] = adjusted_image

In [86]:
import tifffile

tifffile.imwrite("fused_image.tif", fused_image)

In [10]:
frame_1.shape

(1080, 1920, 3)

In [13]:
from scipy.ndimage import affine_transform

angle = 45
theta = np.deg2rad(angle)
x_off = 500
y_off = 0

hom_matr = np.array([
    [np.cos(theta), -np.sin(theta), x_off],
    [np.sin(theta), np.cos(theta), y_off],
    [0, 0, 1]
])

frame_0 = video_subsample[0]
frame_1 = video_subsample[1]
frame_1_adj = np.zeros_like(frame_1)

for i in range(frame_1.shape[2]):
    frame_1_adj[:, :, i] = affine_transform(frame_1[:, :, i], hom_matr)


In [14]:
import napari

viewer = napari.Viewer()

# viewer.add_image(frame_0)
viewer.add_image(frame_1_adj)
viewer.add_image(frame_1)

<Image layer 'frame_1' at 0x3700e1e20>